In [19]:

from .keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
import pandas as pd
from requests import Request, Session, Response
from typing import Optional, Dict, Any, List


class FtxClient:


    def __init__(self, api_key=None, api_secret=None, subaccount_name=None):
            self._session = Session()
            self._api_key = api_key
            self._api_secret = api_secret
            self._subaccount_name = subaccount_name
            self._ENDPOINT = 'https://ftx.com/api/'

    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
            return self.send_request('GET', path, params=params)

    def sign_request(self, request: Request):
        ts = int(time.time() * 1000)
        request = Request('GET', self._ENDPOINT)
        prepared = request.prepare()
        print(prepared)
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)

    def send_request(self, method: str, path: str, **kwargs):
        request = Request(method, self._ENDPOINT + path, **kwargs)
        self.sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _process_response(self, response: Response):
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    def _get_all_trades(self, market: str, start_time: float = None, end_time: float = None):
        ids = set()
        limit = 100
        results = []
        while True:
            response = self._get(f'markets/{market}/trades', {
                'end_time': end_time,
                'start_time': start_time,
            })
            deduped_trades = [r for r in response if r['id'] not in ids]
            results.extend(deduped_trades)
            ids |= {r['id'] for r in deduped_trades}
            print(f'Adding {len(response)} trades with end time {end_time}')
            if len(response) == 0:
                break
            end_time = min(parse_datetime(t['time']) for t in response).timestamp()
            if len(response) < limit:
                break
        return results


AttributeError: module 'sys' has no attribute 'insert'

In [20]:
import sys
sys.path

['/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/notebooks',
 '/home/harry/.vscode-server/extensions/ms-toolsai.jupyter-2021.3.619093157/pythonFiles/vscode_datascience_helpers/../.does-not-exist',
 '/home/harry/.vscode-server/extensions/ms-toolsai.jupyter-2021.3.619093157/pythonFiles',
 '/home/harry/.vscode-server/extensions/ms-toolsai.jupyter-2021.3.619093157/pythonFiles/lib/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/home/harry/.local/lib/python3.9/site-packages',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/harry/.local/lib/python3.9/site-packages/IPython/extensions',
 '/home/harry/.ipython']

In [2]:
import json

# dt = datetime(2021, 1, 1).timestamp()
acc = FtxClient(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)
a= acc._get('futures')
for ticker in a: 
    print(ticker['underlying'])
# res = acc.send_request('GET', 'markets/BTC-PERP/candles?resolution=15&limit=10')
# all_trade = acc._get_all_trades('BTC-PERP', start_time='1559881511', end_time='1609477200')
# df = pd.DataFrame(res)
# print(df)

NameError: name 'FtxClient' is not defined

In [9]:
from datetime import datetime

dt = datetime(2021, 2, 1).timestamp()
unix = datetime.fromtimestamp(1606798800)
print(unix)

2021-03-07 00:00:01
